instalamos  paquetes

In [1]:
import pandas as pd
from geopy.distance import geodesic
import uuid
import heapq
import folium
import json

In [2]:
# Leer el archivo CSV
data = pd.read_csv('data.csv')

# Crear un diccionario para almacenar la información de los aeropuertos
airports = {}
flights = {} 

# Iterar sobre las filas del DataFrame y almacenar la información en el diccionario
for index, row in data.iterrows():
    source_coords = (row['Source Airport Latitude'], row['Source Airport Longitude'])
    dest_coords = (row['Destination Airport Latitude'], row['Destination Airport Longitude'])

    distance = geodesic(source_coords, dest_coords).kilometers
    code1 = str(uuid.uuid4())
    flight_info = {
        'source': row['Source Airport Code'],
        'destination': row['Destination Airport Code'],
        'distance': distance
    }

    flights[code1] = flight_info

    code2 = str(uuid.uuid4())
    flight_info = {
        'source': row['Destination Airport Code'],
        'destination': row['Source Airport Code'],
        'distance': distance
    }

    flights[code2] = flight_info

    airports[row['Source Airport Code']] = {
        'name': row['Source Airport Name'],
        'city': row['Source Airport City'],
        'country': row['Source Airport Country'],
        'coordinates': source_coords
    }
    
    airports[row['Destination Airport Code']] = {
        'name': row['Destination Airport Name'],
        'city': row['Destination Airport City'],
        'country': row['Destination Airport Country'],
        'coordinates': dest_coords
    }

In [3]:
airport_objects = [{'code': code, 'name': info['name'], 'city': info['city'], 'country': info['country'], 'coordinates': [info['coordinates'][0], info['coordinates'][1]]} for code, info in airports.items()]
flights_objects = [{'code': code, 'source': info['source'], 'destination': info['destination'], 'distance': info['distance']} for code, info in flights.items()]

def eliminar_duplicados(flights_objects):
    # Creamos un conjunto para almacenar las tuplas únicas (source, destination, distance)
    unique_entries = set()
    # Lista para almacenar las entradas únicas
    unique_flights_objects = []
    
    for flight in flights_objects:
        # Creamos una tupla con las claves relevantes para comparar
        entry_tuple = (flight['source'], flight['destination'], flight['distance'])
        # Verificamos si la tupla ya está en el conjunto de entradas únicas
        if entry_tuple not in unique_entries:
            # Si no está, agregamos la entrada a la lista de entradas únicas
            unique_flights_objects.append(flight)
            # Agregamos la tupla al conjunto de entradas únicas
            unique_entries.add(entry_tuple)
    
    return unique_flights_objects

flights_objects = eliminar_duplicados(flights_objects)

In [4]:
#clase para gestionar la informacion de aeropuertos y vuelos 
class AirportFlightsObject:
    def __init__(self, airports, flights):
        #graph (dict): Un diccionario que almacena la información de aeropuertos y conexiones de vuelo. Cada aeropuerto es un nodo, y las conexiones de vuelo son los bordes en el grafo.
        self.graph = {}
        #- Para cada elemento en el diccionario 'airports', donde 'code' es la clave (código del aeropuerto) y 'info' es el valor (información del aeropuerto), se crea una entrada en 'self.graph'. Esta entrada contiene:
        for code, info in airports.items():
            self.graph[code] = {'name': info['name'], 'city': info['city'], 'country': info['country'], 'coordinates': info['coordinates'], 'connections': {}}
        for code, info in flights.items():
            #- Para cada elemento en el diccionario 'flights', donde 'code' es la clave (código del aeropuerto) y 'info' es el valor (información del aeropuerto), se crea una entrada en 'self.graph'. Esta entrada contiene:
            source = info['source']
            destination = info['destination']
            distance = info['distance']
            self.graph[source]['connections'][destination] = distance
    
    def dijkstra(self, start_node):
        # 1. Inicializa un diccionario para el seguimiento de las distancias más cortas.
        distances = {node: {'distance': float('infinity'), 'last_node': None} for node in self.graph}
        # 2. Establece la distancia del nodo de inicio en 0 y lo agrega a la cola de prioridad.
        distances[start_node]['distance'] = 0
        priority_queue = [(0, start_node)]

        # 3. Bucle principal: Procesa nodos en la cola de prioridad.
        while priority_queue:
            current_distance, current_node = heapq.heappop(priority_queue)
            # 4. Si la distancia actual es mayor que la registrada en 'distances', continúa al siguiente nodo.
            if current_distance > distances[current_node]['distance']:
                continue
            # 5. Calcula y actualiza las distancias a los nodos vecinos.    
            for last_node, weight in self.graph[current_node]['connections'].items():
                distance = current_distance + weight
                if distance < distances[last_node]['distance']:
                    distances[last_node]['distance'] = distance
                    distances[last_node]['last_node'] = current_node
                    heapq.heappush(priority_queue, (distance, last_node))
        # 6. Recopila información sobre las rutas más cortas y retorna el resultado.            
        shortest_paths = []
        for node, info in distances.items():
            if info['distance'] != float('infinity'):
                shortest_paths.append({
                    'code': node,
                    'distance': info['distance'],
                    'last_node': info['last_node']
                })
        return shortest_paths
        
    def get_airport_info(self, airport_code):
        # Verifica si el aeropuerto está en el grafo.
        if airport_code in self.graph:
            # Recupera la información del aeropuerto y la retorna como un diccionario.
            airport_info = self.graph[airport_code]
            return {
                'name': airport_info['name'],
                'city': airport_info['city'],
                'country': airport_info['country'],
                'coordinates': [airport_info['coordinates'][0], airport_info['coordinates'][1]],
                'connections': airport_info['connections'],
                'dijkstra': self.dijkstra(airport_code)
            }
        else:
            # Retorna un mensaje de "Aeropuerto no encontrado" si el código de aeropuerto no existe en el grafo.
            return "Aeropuerto no encontrado"
    
    def get_larger_paths(self, airport_code): #Definimos una funcion para conocer los 10 caminos mas largos desde el aeropuerto seleccionado
        paths = airport_flights_object.dijkstra(airport_code)   #Obtenemos el Dijkstra
        ordenated_paths = sorted(paths, key = lambda x:x['distance'], reverse = True)  #Organizamos la lista del mayor al menos usando como llave las distancias
        larger_paths = ordenated_paths[:10]  #Seleccionamos los 10 primeros datos
        larger_paths_info = []
        for paths in larger_paths:
            info = airport_flights_object.get_airport_info(paths['code']) #Utilizamos la funcion get_airport_info para obtener la informacion necesaria de cada uno de los aeropuertos
            code = paths['code']
            distance = paths['distance']
            name = info['name']
            city = info['city']
            country = info['country']
            location = info['coordinates']
            information = {'code': code,'name': name, 'city': city,'country': country, 'distance': distance, 'location': location}
            larger_paths_info.append(information) #Guardamos toda la informacion en una lista de diccionarios
        return larger_paths_info
        #print(larger_paths_info)

In [5]:
# Ejemplo de us# airports y flights son los diccionarios que tienes con la información de aeropuertos y vuelos respectivamente
airport_flights_object = AirportFlightsObject(airports, flights)
start_node = input("Ingrese el codigo del aeropuerto del cual quiera conocer sus caminos minimos: ") # Ingrese el codigo del aeropuerto del cual desee conocer su dijkstra
shortest_paths = airport_flights_object.dijkstra(start_node)
print(shortest_paths)

[{'code': 'COK', 'distance': 15550.608136908275, 'last_node': 'MAA'}, {'code': 'SHJ', 'distance': 13487.23593870998, 'last_node': 'IKA'}, {'code': 'GIG', 'distance': 10130.434311029523, 'last_node': 'PTY'}, {'code': 'BSB', 'distance': 9275.291105810305, 'last_node': 'PTY'}, {'code': 'TSF', 'distance': 9916.946791896795, 'last_node': 'CRL'}, {'code': 'BRI', 'distance': 10516.79253705016, 'last_node': 'AMS'}, {'code': 'CDG', 'distance': 9124.094346132713, 'last_node': 'LAX'}, {'code': 'FLR', 'distance': 10004.756254515492, 'last_node': 'CDG'}, {'code': 'CGH', 'distance': 10144.149320878678, 'last_node': 'BSB'}, {'code': 'CGB', 'distance': 9274.520174732144, 'last_node': 'PVH'}, {'code': 'DFW', 'distance': 1987.3829886166982, 'last_node': 'LAX'}, {'code': 'YUL', 'distance': 3980.854735938383, 'last_node': 'LAX'}, {'code': 'DOM', 'distance': 6043.963674428948, 'last_node': 'SJU'}, {'code': 'PTP', 'distance': 5982.0623178102105, 'last_node': 'MIA'}, {'code': 'ATL', 'distance': 3132.51945238

In [6]:
map_center = [0, 0]
mapa = folium.Map(location=map_center, zoom_start=2)

for airport in airport_objects:
    popup_content = f"code: {airport['code']}, name: {airport['name']}, city: {airport['city']}, country: {airport['country']}, coordinates: {airport['coordinates']}"
    folium.Marker(location = airport['coordinates'], popup = popup_content).add_to(mapa)

mapa.save('aeropuertos.html')

In [7]:
# Mostrar la información correspondiente al aeropuerto (código, nombre,ciudad, país, latitud y longitud).
airport_code = input("Ingrese el codigo del aeropuerto que desee conocer: ") # Código del aeropuerto que deseas buscar
airport_info = airport_flights_object.get_airport_info(airport_code)
print(airport_info)

{'name': 'El Dorado International Airport', 'city': 'Bogota', 'country': 'Colombia', 'coordinates': [4.70159, -74.1469], 'connections': {'CLO': 279.2351005521599, 'DFW': 3924.087054999436, 'MEX': 3157.5497588357875, 'GIG': 4533.5919059434445, 'HAV': 2209.9965836145166, 'CUC': 400.0119453802294, 'FRA': 9089.635878151126, 'AUA': 975.0343585469006, 'GRU': 4326.962202307335, 'CUR': 1005.7428950617062, 'CTG': 652.539468999751, 'JFK': 3980.85697660289, 'PSO': 505.7156656865163, 'BGA': 288.5454417776721, 'CCS': 1023.9452117983568, 'MTR': 492.18855045510014, 'EOH': 231.96713751720077, 'MIA': 2424.3089578782096, 'SJE': 288.2711125470629, 'IAH': 3576.9070600112746, 'PUU': 532.7352013317077, 'SCL': 4231.091104111378, 'GUA': 2101.8414081482983, 'CUN': 2271.5634696048483, 'AUC': 459.4597342852342, 'FLA': 378.5058866940526, 'IBE': 113.75425690180008, 'IPI': 577.7716819629486, 'MDE': 214.88233103533423, 'YYZ': 4351.611883571412, 'PPN': 369.5749197440971, 'UIO': 710.3236315798624, 'LET': 1089.03434650

In [8]:
# Mostrar la información (código, nombre, ciudad, país, latitud y longitud) de los 10 aeropuertos cuyos caminos mínimos desde el vértice dado sean los más largos. Adicionalmente, se debe mostrar la distancia de los caminos.
start_node = input("Ingrese el nodo del cual quiera conocer los 10 caminos mas largos: ") 
ten_paths = airport_flights_object.get_larger_paths(start_node) #Utilizamos la funcion get_larger_paths para obtener los 10 caminos mas largos del dijkstra del aeropuerto seleccionado
print(ten_paths)

map_center = [0, 0]
mapa = folium.Map(location=map_center, zoom_start=2)

for airport in ten_paths:
    popup_content = f"code: {airport['code']}, name: {airport['name']}, city: {airport['city']}, country: {airport['country']}, coordinates: {airport['location']}"
    folium.Marker(location = airport['location'], popup = popup_content).add_to(mapa)

mapa.save('10_aeropuertos_con_los_caminos_mas_largos.html')

[{'code': 'IPC', 'name': 'Mataveri Airport', 'city': 'Easter Island', 'country': 'Chile', 'distance': 19640.09944975772, 'location': [-27.16480064, -109.4219971]}, {'code': 'MPN', 'name': 'Mount Pleasant Airport', 'city': 'Mount Pleasant', 'country': 'Falkland Islands', 'distance': 19245.715815802672, 'location': [-51.82279968, -58.44720078]}, {'code': 'TBP', 'name': 'Capitan FAP Pedro Canga Rodriguez Airport', 'city': 'Tumbes', 'country': 'Peru', 'distance': 18961.543369194853, 'location': [-3.55253005, -80.38140106]}, {'code': 'PIU', 'name': 'Capitán FAP Guillermo Concha Iberico International Airport', 'city': 'Piura', 'country': 'Peru', 'distance': 18804.6545961432, 'location': [-5.205749989, -80.61640167]}, {'code': 'USH', 'name': 'Malvinas Argentinas Airport', 'city': 'Ushuaia', 'country': 'Argentina', 'distance': 18785.053863397487, 'location': [-54.8433, -68.2958]}, {'code': 'BBA', 'name': 'Balmaceda Airport', 'city': 'Balmaceda', 'country': 'Chile', 'distance': 18725.2861647231

In [9]:
# Mostrar el camino mínimo entre el primer y el segundo vértice sobre el mapa de la interfaz gráfica, pasando por cada uno de los vértices intermedios del camino. Para cada vértice intermedio se debe mostrar la información del aeropuerto (código, nombre, ciudad, país, latitud y longitud).
mapa2 = folium.Map(location=[0, 0], zoom_start=2)
start_node = input("Ingrese el aeropuerto de inicio: ")
final_node = input("Ingrese el aeropuerto destino: ")

selected_airports = []
last_airport = None
continues = 1
for airports in airport_flights_object.dijkstra(start_node): #Recorremos el dijkstra del aeropuerto inicial
        if airports['code'] == final_node:
            selected_airports.append(airports) #Guardamos la informacion del aeropuerto destino buscada en el dijkstra
            last_airport = airports['last_node']  #Guardamos el aeropuerto anterior para buscar el camino minimo
while continues == 1:
    for airports in airport_flights_object.dijkstra(start_node):
        if airports['code'] == last_airport and airports['code'] == start_node: #Recorremos el dijkstra y si encontramos el aeropuerto inicial el camino habra terminado
            selected_airports.append(airports)
            continues = 0
        if airports['code'] == last_airport and continues == 1: #Recorremos el dijkstra en el caso que todavia no hayamos encontrado el aeropuerto inicial y vamos agregando el camino para llegar a el
            selected_airports.append(airports)
            last_airport = airports['last_node']

airports_info = []
for paths in selected_airports:
    info = airport_flights_object.get_airport_info(paths['code']) #Obtenemos la informacion de cada uno de los aeropuertos
    code = paths['code']
    name = info['name']
    city = info['city']
    country = info['country']
    location = info['coordinates']
    popup_content = f"code: {paths['code']}, name: {info['name']}, city: {info['city']}, country: {info['country']}, coordinates: {info['coordinates']}"
    folium.Marker(location = info['coordinates'],popup = popup_content).add_to(mapa2)
    airports_infos = {'code': code, 'name': name, 'city': city, 'country': country, 'location': location}
    airports_info.append(airports_infos)
print(airports_info) #Esta lista mostrara los datos obtenidos de cada uno de los aeropuertos

ruta_coords = []
for airport in airports_info:
    ruta_coords.append(airport['location']) #Guardamos cada una de las coordenadas de los aeropuertos de la lista

folium.PolyLine(locations=ruta_coords, color="blue", weight=2.5, opacity=1.0).add_to(mapa2)
mapa2.save("aeropuertos_con_ruta.html") #Mostramos en la interfaz el camino a seguir para llegar desde el aeropuerto inicial hasta el aeropuerto destino

[{'code': 'GOH', 'name': 'Godthaab / Nuuk Airport', 'city': 'Godthaab', 'country': 'Greenland', 'location': [64.19090271, -51.67810059]}, {'code': 'KEF', 'name': 'Keflavik International Airport', 'city': 'Keflavik', 'country': 'Iceland', 'location': [63.98500061, -22.60560036]}, {'code': 'GLA', 'name': 'Glasgow International Airport', 'city': 'Glasgow', 'country': 'United Kingdom', 'location': [55.87189865, -4.433060169]}, {'code': 'DUB', 'name': 'Dublin Airport', 'city': 'Dublin', 'country': 'Ireland', 'location': [53.421299, -6.27007]}, {'code': 'LIS', 'name': 'Humberto Delgado Airport (Lisbon Portela Airport)', 'city': 'Lisbon', 'country': 'Portugal', 'location': [38.7813, -9.13592]}, {'code': 'BSB', 'name': 'Presidente Juscelino Kubistschek International Airport', 'city': 'Brasilia', 'country': 'Brazil', 'location': [-15.86916733, -47.92083359]}]


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=18f3ef2f-dd46-45ab-a7b1-7ba94723aaf9' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>